Imports

In [20]:
import pandas as pd 
import numpy as np
import os
import datetime
import re

Load files

In [21]:
bandwidths = ['No', '1Gbs'] 
nodes = [4, 8, 16] 
replicates_number = 2

df_results = pd.DataFrame()
prefix_path = "/home/ninon/description_prototype_stuff/merging_results/"
reports_files = []

for i in range(len(bandwidths)):
    for j in range(len(nodes)):
        for k in range(replicates_number):
            mode = "baseline"
            path = prefix_path +  str(bandwidths[i]) + "/"+ str(nodes[j]) + "/" + mode + "/" +  str(k+1)
            prefix = "_trace"
            #print(path)
            if os.path.exists(path):
                csv_files = [file for file in os.listdir(path) if file.startswith(prefix)]
                for file in csv_files:
                    file_path = os.path.join(path, file) 
                    df = pd.read_csv(file_path, delimiter="\t")
                    df['bandwidth'] = str(bandwidths[i])
                    df["node"] = str(nodes[j]) 
                    df["mode"] = mode 
                    df["replicate"] =  str(k+1)
                    df_results = df_results.append(df, ignore_index=True)
            else:
                mode = "rewritten"
                path = prefix_path +  str(bandwidths[i]) + "/"+ str(nodes[j]) + "/" + mode + "/" +  str(k+1)
                prefix = "_trace"
                #print(path)
                if os.path.exists(path):
                    csv_files = [file for file in os.listdir(path) if file.startswith(prefix)]
                    for file in csv_files:
                        file_path = os.path.join(path, file) 
                        df = pd.read_csv(file_path, delimiter="\t")
                        df['bandwidth'] = str(bandwidths[i])
                        df["node"] = str(nodes[j]) 
                        df["mode"] = mode 
                        df["replicate"] =  str(k+1)
                        df_results = df_results.append(df, ignore_index=True)
                else:
                    #print("no")
                    continue

print(df_results.head())

# Find NaN values
nan_rows = df[df.isna().any(axis=1)]
nan_counts_per_row = nan_rows.isna().sum(axis=1)

print("Rows with NaN values:")
print(nan_rows)
print("\nNumber of NaN values in each row:")
print(nan_counts_per_row)

        hostname  task_id       hash                            native_id  \
0  hu-worker-c27        5  90/a3cc1f  nf-90a3cc1f2cdfc19b35a9cdbdae48cc34   
1  hu-worker-c24        3  f2/938a2e  nf-f2938a2e977b8b24b5ee4a3f64ef4059   
2  hu-worker-c25        4  44/547e6d  nf-44547e6d76ad212a46f916b5cecc6361   
3  hu-worker-c24        1  8c/886f80  nf-8c886f8078e73853475a073d52db85e6   
4  hu-worker-c26        2  c3/1920aa  nf-c31920aa54f83f03f90353244aa2755d   

                 process                                       tag  \
0        GFFREAD_TX2GENE  Drosophila_melanogaster.BDGP6.32.106.gtf   
1   CUSTOM_GETCHROMSIZES                                         -   
2      DEXSEQ_ANNOTATION  Drosophila_melanogaster.BDGP6.32.106.gtf   
3  SALMON_GENOMEGENERATE                                         -   
4    STAR_GENOMEGENERATE                                         -   

                                                name     status  exit module  \
0  GFFREAD_TX2GENE (Drosophila_melan

Compute runtime for each experiment and compare baseline vs rewritten

In [47]:

def convert_to_timedelta(time_string):
    components = {'h': 'hours', 'm': 'minutes', 's': 'seconds'}
    time_parts = {components[part[-1]]: int(part[:-1]) for part in re.findall(r'\d+[hms]', time_string)}
    timedelta= datetime.timedelta(**time_parts)
    return timedelta

def convertStrToTimeFromReport(mstr):
    runtime = mstr.split("</strong>")[-2]
    runtime = runtime.split("<strong>")[1]
    runtime = convert_to_timedelta(runtime)
    return runtime

def fetchRunTimeFromReport(file_path_list):
    if( len(file_path_list) < 2 ):
        with open(file_path_list[0], 'r') as file:
            for line in file:
                if ('completed_fromnow"></span>duration: <strong>' in line):
                    runtime = convertStrToTimeFromReport(line)
                    break 
    else :
        with open(file_path_list[0], 'r') as file:
            for line in file:
                if ('completed_fromnow"></span>duration: <strong>' in line):
                    runtime_1 = convertStrToTimeFromReport(line)
                    break 
        with open(file_path_list[1], 'r') as file:
            for line in file:
                if ('completed_fromnow"></span>duration: <strong>' in line):
                    runtime_2 = convertStrToTimeFromReport(line)
                    break 
        runtime = (runtime_1 + runtime_2) / 2
    return runtime 

def computeDiffInRuntime(baseline, rewritten):
    abs_diff = abs(baseline - rewritten)
    percent_diff = (abs_diff / baseline) * 100
    return abs_diff, percent_diff


df_runtime = pd.DataFrame()

prefix = "_report"

for i in range(len(bandwidths)):
    for j in range(len(nodes)):

        #get baseline runtime values

        mode = "baseline"
        path = prefix_path +  str(bandwidths[i]) + "/"+ str(nodes[j]) + "/" + mode + "/" +  str(1)
        path_replicate = prefix_path +  str(bandwidths[i]) + "/"+ str(nodes[j]) + "/" + mode + "/" +  str(1)
        exp_name = str(bandwidths[i]) + "_band_" + str(nodes[j]) + "_nodes"

        #check if replicate present
        if (os.path.exists(path) and os.path.exists(path_replicate)):
            # deal with average btw 2 runs
            file = [file for file in os.listdir(path) if file.startswith(prefix)][0]
            file_path = os.path.join(path, file) 
            file = [file for file in os.listdir(path_replicate) if file.startswith(prefix)][0]
            file_path_replicate = os.path.join(path_replicate, file) 
            avg_runtime_baseline = fetchRunTimeFromReport([file_path, file_path_replicate])
            baseline_runtime = avg_runtime_baseline
        #else just one file is present
        elif os.path.exists(path):
            file = [file for file in os.listdir(path) if file.startswith(prefix)][0]
            file_path = os.path.join(path, file) 
            runtime = fetchRunTimeFromReport([file_path])
            baseline_runtime = runtime
        else:
            continue

        # get rewritten runtime values

        mode = "rewritten"
        path_rewritten = prefix_path +  str(bandwidths[i]) + "/"+ str(nodes[j]) + "/" + mode + "/" +  str(1)
        path_replicate_rewritten = prefix_path +  str(bandwidths[i]) + "/"+ str(nodes[j]) + "/" + mode + "/" +  str(2)

        # if replicate present
        if (os.path.exists(path_rewritten) and os.path.exists(path_replicate_rewritten)):
            file_rewritten = [file for file in os.listdir(path_rewritten) if file.startswith(prefix)][0]
            file_path_rewritten = os.path.join(path_rewritten, file_rewritten) 
            file_rewritten = [file for file in os.listdir(path_replicate_rewritten) if file.startswith(prefix)][0]
            file_path_replicate_rewritten = os.path.join(path_replicate_rewritten, file_rewritten) 
            avg_runtime_rewritten = fetchRunTimeFromReport([file_path_rewritten, file_path_replicate_rewritten])
            rewritten_runtime = avg_runtime_rewritten
        
        #else
        elif os.path.exists(path_rewritten):
            file_rewritten = [file for file in os.listdir(path_rewritten) if file.startswith(prefix)][0]
            file_path_rewritten = os.path.join(path_rewritten, file_rewritten)
            runtime = fetchRunTimeFromReport([file_path_rewritten])
            rewritten_runtime = runtime
        else:
            continue

        timediff_min, timediff_per = computeDiffInRuntime(baseline_runtime, rewritten_runtime)
        new_row = {'exp': exp_name, 'baseline runtime': baseline_runtime, 'rewritten runtime': rewritten_runtime, 'timediff_min': timediff_min, 'timediff_per':timediff_per}
        df_runtime = df_runtime.append(new_row, ignore_index=True)

print(df_runtime)


7:16:03
72.09622750695803
7:20:49.500000
73.09722529294716
7:24:04.500000
73.78908305408623
7:21:00
72.78428783627662
7:18:12
73.4598083317035
7:19:21
73.75975824729288
  baseline runtime                 exp      rewritten runtime  \
0  0 days 10:04:49     No_band_4_nodes        0 days 02:48:46   
1  0 days 10:03:04     No_band_8_nodes 0 days 02:42:14.500000   
2  0 days 10:01:49    No_band_16_nodes 0 days 02:37:44.500000   
3  0 days 10:05:54   1Gbs_band_4_nodes        0 days 02:44:54   
4  0 days 09:56:31   1Gbs_band_8_nodes        0 days 02:38:19   
5  0 days 09:55:39  1Gbs_band_16_nodes        0 days 02:36:18   

            timediff_min  timediff_per  
0        0 days 07:16:03     72.096228  
1 0 days 07:20:49.500000     73.097225  
2 0 days 07:24:04.500000     73.789083  
3        0 days 07:21:00     72.784288  
4        0 days 07:18:12     73.459808  
5        0 days 07:19:21     73.759758  
